<a href="https://colab.research.google.com/github/ICBI/Data.Bridge.Notebooks/blob/main/Goal1_SDOH/Tutorial1_BigQueryPythonGuide.ipynb/Tutorial1_BigQueryPythonGuide.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Tutorial: How to query the Georgetown-SDOH BigQuery datawarehouse using python

**Team members**: Peter McGarvey, Adil Alaoui, Yili Zhang, Jia Li Dong, Krithika Bhuvaneshwar, Camelia Bencheqroun

**Affiliation**: Innovation Center for Biomedical Informatics (Georgetown-ICBI), Georgetown University Medical Center. https://icbi.georgetown.edu

## Pre-requisites
* Users must have a google based account, with the ability to use its cloud
* Access/Credentials: Please contact icbi@georgetown.edu to request read-only access to the Georgetown-SDOH BigQuery database

## Installation and Authentication
We need to install the Google Cloud Bigquery Python Client, and complete authentication so that we can connect to the BigQuery data warehouse. So authenticate and log in using your google account. Click **Allow** and log in according to the prompt.

In [ ]:
from google.colab import auth
auth.authenticate_user()
print('Authenticated')

Authenticated


Google Colab includes the `google.colab.data_table` package that can be used to display large pandas dataframes as an interactive data table. It can be enabled as follows:

In [ ]:
%load_ext google.colab.data_table

Declare the Cloud project ID which will be used throughout this notebook. The project ID of the Georgetown-SDOH database is **sdoh-352614**.


In [ ]:
project_id = 'sdoh-352614'

Now, you can query the Georgetown-SDOH BigQuery using SQL !

The beginning format of your query calls for the client, which connects you to the project that you are working on.


In [ ]:
#import the right packages
from google.cloud import bigquery

#create a client object using the project id
client = bigquery.Client(project=project_id)

## Example query: List all the datasets in the Georgetown-SDOH data warehouse





In [ ]:

datasets = list(client.list_datasets())  # Make an API request.
project = client.project

if datasets:
    print("Datasets in project {}:".format(project))
    for dataset in datasets:
        print("\t{}".format(dataset.dataset_id))
        tables = client.list_tables(dataset.dataset_id)

        print("Tables contained in '{}':".format(dataset.dataset_id))
        for table in tables:
            print("{}.{}.{}".format(table.project, table.dataset_id, table.table_id))

else:
    print("{} project does not contain any datasets.".format(project))

Datasets in project sdoh-352614:
	PEDS_DEID
Tables contained in 'PEDS_DEID':
sdoh-352614.PEDS_DEID.Billing Data DEID Mar 15 2022
sdoh-352614.PEDS_DEID.Pediatric Survivorhip GUH Tumor Registry data deid
sdoh-352614.PEDS_DEID.Pediatric Survivorhip GUH data deid with age at dx without dates
sdoh-352614.PEDS_DEID.Pediatric Survivorhip GUH data deid with age at dx without dates with FIPS code
	Social_Determinants
Tables contained in 'Social_Determinants':
sdoh-352614.Social_Determinants. USCS BySite 1999-2019
sdoh-352614.Social_Determinants. USCS ChildByAge Adjusted 1999-2019
sdoh-352614.Social_Determinants. USCS ChildByAge Cr 1999-2019
sdoh-352614.Social_Determinants. USCS ChildBySite 1999-2019
sdoh-352614.Social_Determinants. USCS ICCCByAge Adjusted 1999-2019
sdoh-352614.Social_Determinants. USCS ICCCByAge Cr 1999-2019
sdoh-352614.Social_Determinants. USCS ICCCBySite 1999-2019
sdoh-352614.Social_Determinants.2015 County Health Data
sdoh-352614.Social_Determinants.2018 County Health Additi

## Example 1: Load the ***Eviction State Monthly Data*** from the SDoH datawarehouse into a new dataframe in Colab called **evicstmth**.

We use SQL to access the database using the client. Use the SQL syntax of ```SELECT * FROM the_location_of_your_project.the_location_of_your_dataset.the_table_location``` **inside** the client query with three single quotation marks in order to obtain the data.



Say for example, you wanted to query into a table in BigQuery named **SVI 2018**, which is inside the BigQuery dataset named **Social_Determinants**. The **Social_Determinants** dataset is inside a BigQuery project named **sdoh-352614**. You wanted to load the data table into a new dataframe in Colab called **df1**. You will type this inside the client query:

```SELECT * FROM `sdoh-352614.Social_Determinants.SVI 2018```

Then store the client query table inside the new dataframe called **df1**.

```df1 = client.query('''
SELECT * FROM `sdoh-352614.Social_Determinants.SVI 2018`
''').to_dataframe()```

In [ ]:
#%%bigquery --project sdoh-352614 evicstmth

query1 = '''SELECT * FROM `sdoh-352614.Social_Determinants.Eviction State Monthly Data`'''
evicstmth = client.query(query1).to_dataframe()

View the first five rows:

In [ ]:
evicstmth.head()

,state,type,GEOID,month,filings_2020,filings_avg,last_updated
0,delaware,Census Tract,10003016305,22-Jun,11,0.0,2022-06-04
1,indiana,Census Tract,18077966100,21-Mar,13,0.0,2022-06-04
2,indiana,Census Tract,18077966100,21-May,10,0.0,2022-06-04
3,indiana,Census Tract,18077966100,21-Jul,10,0.0,2022-06-04
4,indiana,Census Tract,18077966100,21-Sep,10,0.0,2022-06-04


## Example 2: Load another SDoH dataset, Neighborhood Deprivation Index with data from from 2013 to 2017

Load another SDoH dataset, Neighborhood Deprivation Index, from 2013 to 2017, known as the ***NDI 2013-2017***, into another dataframe in Colab and name it as **NDI**.

In [ ]:
#%%bigquery --project sdoh-352614 NDI
query2 = '''SELECT *FROM `sdoh-352614.Social_Determinants.NDI 2013-2017`'''
NDI = client.query(query2).to_dataframe()

As you can see, after connecting to BigQuery, we have to query the dataset using SQL. In the **FROM** section of SQL, we use the following format:
```project_location```.```dataset_location```.```table_name```

We first type in the project location, followed by a period, then the dataset location, followed by another period, and lastly, the table name.

As quotation marks, we use the squiggly line key, which is between the Esc key and the Tab key.

Here is the NDI dataset in Python that we've queried in the previous section:

In [ ]:
NDI.head()

,TractID,StCoFIPS,StAbbr,NDI,NDIQuint,MedHHInc,PctRecvIDR,PctPubAsst,MedHomeVal,PctMgmtBusSciArt,PctFemHeadKids,PctOwnerOcc,PctNoPhone,PctNComPlmb,PctEducHSPlus,PctEducBchPlus,PctFamBelowPov,PctUnempl
0,1101003000,1101,AL,NaN,9-NDI not avail,26635,2.5,40.9,62300,NaN,26.1,46.3,2.1,3.5,72.2,11.6,40.3,22.8
1,2016000100,2016,AK,NaN,9-NDI not avail,62083,NaN,1.0,81300,28.1,NaN,60.8,4.2,2.1,92.0,13.8,12.3,3.5
2,2105000200,2105,AK,NaN,9-NDI not avail,42500,65.0,NaN,<NA>,39.3,3.0,75.0,6.3,9.4,83.9,16.1,5.3,12.5
3,2170000101,2170,AK,NaN,9-NDI not avail,37222,66.9,NaN,128100,22.6,6.6,82.4,6.3,22.4,87.8,11.5,22.0,15.8
4,2290000100,2290,AK,NaN,9-NDI not avail,27222,53.4,NaN,81100,38.3,14.2,71.7,7.1,56.4,77.6,8.7,34.7,23.1


Here, we describe the data:

In [ ]:
NDI.describe()

,TractID,StCoFIPS,NDI,MedHHInc,PctRecvIDR,PctPubAsst,MedHomeVal,PctMgmtBusSciArt,PctFemHeadKids,PctOwnerOcc,PctNoPhone,PctNComPlmb,PctEducHSPlus,PctEducBchPlus,PctFamBelowPov,PctUnempl
count,73056.0,73056.0,68729.000000,72010.0,71496.000000,70757.000000,70965.0,71820.000000,70524.000000,72240.000000,71679.000000,72240.000000,72410.000000,72410.000000,72189.000000,72305.000000
mean,27826014983.706402,27825.756324,0.000578,61571.416706,20.630293,14.671076,242698.524625,35.581164,8.626781,63.203056,2.406512,0.433675,86.660500,29.615430,12.014513,7.102571
std,15810103234.821383,15810.097067,0.745852,30358.730222,11.997621,12.233646,206762.842096,15.139815,6.429578,22.808354,2.471933,1.283982,10.761807,19.013141,11.075066,4.955727
min,1001020100.0,1001.0,-2.470200,3709.0,0.100000,0.000000,10400.0,0.900000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,12127083004.5,12127.0,-0.469700,40781.0,11.600000,5.600000,111800.0,24.700000,4.100000,48.600000,0.900000,0.000000,81.900000,14.900000,4.100000,3.900000
50%,27123041601.5,27123.0,0.084500,54766.0,19.100000,11.300000,173300.0,33.400000,6.900000,68.200000,1.900000,0.000000,89.600000,24.600000,8.600000,5.900000
75%,41035971225.0,41035.0,0.527400,75000.0,27.600000,20.300000,300900.0,45.000000,11.500000,81.200000,3.200000,0.500000,94.400000,40.700000,16.700000,8.900000
max,56045951300.0,56045.0,1.922000,249750.0,94.000000,92.300000,1998300.0,92.000000,89.800000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000,100.000000


## Example 3 : Access the FBI Crime data from 2019. Then sort based on an attribute of interest

In [ ]:
query3 = '''SELECT * FROM `sdoh-352614.Social_Determinants.FBI Crime Data 2019` LIMIT 1000'''
fbicrime = client.query(query3).to_dataframe()
fbicrime.head()

,Geo_FIPS,Geo_NAME,Geo_QNAME,Geo_NATION,Geo_STATE,Geo_COUNTY,SE_T001_001,SE_T002_001,SE_T002_002,SE_T002_003,...,SE_T010_003,SE_T010_004,SE_T010_005,SE_T010_006,SE_T011_001,SE_T011_002,SE_T011_003,SE_T011_004,SE_T011_005,SE_T011_006
0,4009,Graham County,"Graham County, Arizona",0,4,9,38837,165,61,104,...,None,None,None,None,None,None,None,None,None,None
1,4015,Mohave County,"Mohave County, Arizona",0,4,15,212181,2118,135,1983,...,None,None,None,None,None,None,None,None,None,None
2,5063,Independence County,"Independence County, Arkansas",0,5,63,37825,520,153,367,...,None,None,None,None,None,None,None,None,None,None
3,6001,Alameda County,"Alameda County, California",0,6,1,1671329,2802,633,2169,...,None,None,None,None,None,None,None,None,None,None
4,6015,Del Norte County,"Del Norte County, California",0,6,15,27812,508,84,424,...,None,None,None,None,None,None,None,None,None,None


In [ ]:
#sort based on Geographical state in descending order
fbicrime.sort_values("Geo_STATE", ascending=False)

,Geo_FIPS,Geo_NAME,Geo_QNAME,Geo_NATION,Geo_STATE,Geo_COUNTY,SE_T001_001,SE_T002_001,SE_T002_002,SE_T002_003,...,SE_T010_003,SE_T010_004,SE_T010_005,SE_T010_006,SE_T011_001,SE_T011_002,SE_T011_003,SE_T011_004,SE_T011_005,SE_T011_006
81,54039,Kanawha County,"Kanawha County, West Virginia",0,54,39,178124,1741,380,1361,...,None,None,None,None,None,None,None,None,None,None
77,53025,Grant County,"Grant County, Washington",0,53,25,97733,932,95,837,...,None,None,None,None,None,None,None,None,None,None
78,53033,King County,"King County, Washington",0,53,33,2252782,4228,607,3621,...,None,None,None,None,None,None,None,None,None,None
79,53053,Pierce County,"Pierce County, Washington",0,53,53,904980,7974,1152,6822,...,None,None,None,None,None,None,None,None,None,None
80,53061,Snohomish County,"Snohomish County, Washington",0,53,61,822083,4863,535,4328,...,None,None,None,None,None,None,None,None,None,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
133,1057,Fayette County,"Fayette County, Alabama",0,1,57,16302,<NA>,<NA>,<NA>,...,None,None,None,None,None,None,None,None,None,None
134,1059,Franklin County,"Franklin County, Alabama",0,1,59,31362,<NA>,<NA>,<NA>,...,None,None,None,None,None,None,None,None,None,None
135,1061,Geneva County,"Geneva County, Alabama",0,1,61,26271,<NA>,<NA>,<NA>,...,None,None,None,None,None,None,None,None,None,None
136,1063,Greene County,"Greene County, Alabama",0,1,63,8111,<NA>,<NA>,<NA>,...,None,None,None,None,None,None,None,None,None,None


**Congratulations ! You have now completed this tutorial !**





